<img src="yt_logo.png" width=120/>

# An introduction to data visualization with yt

[`yt`](https://yt-project.org) is a Python library that provides a generic interface for analysis and visualization of spatially-organised data, with built-in compatibility for various data formats ([Athena++, RAMSES, AREPO, to name a few](https://yt-project.org/doc/examining/index.html)).
It supports grid-based data (regular, stretched, and various flavours of AMR), as well as particle data (point-like and SPH).
Its key features are:
- a code-agnostic interface (specificities of data formats are rarely relevant)
- dimensionally meaningful analysis
- lazy data loading (only load in memory what's actually needed, when it's needed)
- parallel capacity (with `mpi4py`)


In this tutorial, we'll demonstrate how to load data, and produce simple visulations with yt in a couple lines of Python, and show how to go beyond the defaults.
We will cover the following basic functionalities.
- `yt.load`
- `yt.SlicePlot`
- `yt.ProjectionPlot`
- `yt.Dataset.add_field`

Then, we will see how to enable compatibility with Idefix (and Pluto !) data files using the [`yt_idefix`](https://github.com/neutrinoceros/yt_idefix) extension.


For completeness, `yt` supports many operations that will not be covered here, like
- plotting multiple fields with a single `SlicePlot` object
- data reduction to 1D-profile
- "phase plots"
- volume rendering
- off-axis slices and projections
- region selection
- exporting reduced datasets to `HDF5`

*note* that many of these features may have only partial support for non-cartesian geometries.

## Resources
Explore [the documentation](https://yt-project.org/doc/) and [the cookbook](https://yt-project.org/doc/cookbook/index.html) for much more.

- [Homepage](https://yt-project.org/)
- [yt users mailing list](https://mail.python.org/archives/list/yt-users@python.org/)
- [yt Slack Space](https://yt-project.org/slack.html)
- [The `yt_idefix` extension](https://pypi.org/project/yt-idefix/)
- [... and its bug tracker](https://github.com/neutrinoceros/yt_idefix/issues)
- [unyt, a Python library that provides *numpy arrays with units*](https://unyt.readthedocs.io)
- [yt's bug tracker](https://github.com/yt-project/yt/issues)


## 1 - 2D viz with yt

The workflow we'll use here is follows 3 steps
- load our dataset
- create a plot (slice or projection)
- customize and augment the result (yt *plot annotations*)

We'll use a sample dataset provided by `yt` using the `yt.load_sample` function.

In [ ]:
import yt

In [ ]:
ds = yt.load_sample("IsolatedGalaxy")

In [ ]:
# plot a slice of the gas density
yt.SlicePlot(ds, "z", "density")

In [ ]:
# plot the *column* density
yt.ProjectionPlot(ds, "z", "density")

Notice that coordinates and density have units, and that a `yt.ProjectionPlot` effectively
shows a *column* density (or *surface* density), as a sum along the `"z"` direction.
Now let's demonstrate how we can customize the plot. We'll use a `SlicePlot`.

In [ ]:
p = yt.SlicePlot(ds, "z", "density")
p

In [ ]:
# let's zoom in
p.zoom(30)

In [ ]:
# add a timestamp
p.annotate_timestamp(draw_inset_box=True)

In [ ]:
# add stream lines for the velocity field
p.annotate_streamlines("velocity_x", "velocity_y", color="white")

In [ ]:
# change the orientation
p.swap_axes()

In [ ]:
# again
p.flip_horizontal()

## 2 - Inspecting and defining data fields

`yt` has a notion of 'native' VS 'derived' data fields. The former are directly read from disk, while the latter are computed on the fly.
Many fields are often available out-of-the-box ...



In [ ]:
import yt

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030.hierarchy")
ds.field_list

But it's always possible to define more fields at runtime !
Let's see how we would define a simple field representing the gravitational pull of central potential created by $10^6$ solar masses on an Earth-mass test particle as a function of position.

In `yt`, data fields are essentially [*numpy arrays with units*](https://unyt.readthedocs.io/en/stable/). Working with physically meaningful quantities allows `yt` to verify dimensional consistency at runtime. This crucial feature is powered by the `unyt` library. Let's use it here to define the $GM$ constant.

In [ ]:
import unyt as un
import unyt.physical_constants as cst

GM = cst.G * (10**6 * cst.Msun)

# visual dimension analysis
(GM * cst.Mearth / un.AU**2).units.dimensions

In [ ]:
# this cell has a bug, can you spot it ?


def gravitational_pull(field, data):
    # the pull exerted by a central mass of 10**6 Msun on a test mass (Mearth)
    # as a function of the position
    xc, yc, zc = data.ds.domain_center
    length_unit = data["index", "x"].units
    d2 = (
        (data["x"] - xc) ** 2
        + (data["y"] - yc) ** 2
        + (data["z"] - zc) ** 2
        + 1e-16 * length_unit**2  # smoothing around origin
    )
    return GM * un.Mearth / d2**2


ds.add_field(
    ("gas", "gravitational_pull"),
    function=gravitational_pull,
    sampling_type="cell",
    units="N",  # enable dimensional analysis check at definition time
    force_override=True,  # make this cell re-runable
)

yt.SlicePlot(ds, "x", "gravitational_pull")

## 3 - Working with idefix data

We'll need to install the [`yt_idefix` extension](https://pypi.org/project/yt_idefix/) to get compatibility with idefix (and Pluto) data. We'll also download a small idefix dataset.

Note that, as of version `2.2.0`, this extension supports reading data from multiple output formats:
- Idefix and Pluto vtk files
- Idefix dump files
- Pluto XDMF files (requires `h5py`)

In [ ]:
%pip install yt_idefix

In [ ]:
%%sh
if [ ! -d "yt_data" ]
then
    mkdir yt_data
fi

if [ ! -e "yt_data/ThreeLittlePlanets" ]
then
    wget http://use.yt/upload/459d3ef2 -O ThreeLittlePlanets.tar.gz
    tar -xvzf ThreeLittlePlanets.tar.gz
    mv ThreeLittlePlanets yt_data/ThreeLittlePlanets
    rm ThreeLittlePlanets.tar.gz
    cd ..
fi

In [ ]:
import yt

file = "ThreeLittlePlanets/data.0002.vtk"

ds = yt.load(file)
p = yt.SlicePlot(ds, "z", "density")
p.annotate_title("Default behaviour")

A couple notes:
- Most fields are automatically recognized by `yt/yt_idefix`, and translated to a code-agnostic convention (`"RHO"` becomes `"density"`, `"VX2"` becomes `"velocity_y"`, `"velocity_phi"`, `"velocity_theta"` depending on the geometry !). All fields remain accessible under their native names too.
- Default units are shady ! this is because Idefix doesn't have runtime units, so `yt` assumes simple `cgs` everywhere. We can provide actual at post-processing time as

In [ ]:
ds = yt.load(
    file,
    units_override={
        "length_unit": (5.2, "AU"),
        "mass_unit": (1, "Msun"),
    },
)
p = yt.ProjectionPlot(ds, "z", ("gas", "density"))
p.annotate_title("with units override")

... or we can "hide" units (they are still used internally)

In [ ]:
ds = yt.load(
    file,
    unit_system="code",
)
p = yt.ProjectionPlot(ds, "z", ("gas", "density"))
p.annotate_title("with 'code' unit system")

## 4 - Defining new annotation methods

As an example, we'll write a custom `annotate_planets` method that will add markers at planet locations.

In [ ]:
from pathlib import Path

import numpy as np
from yt.visualization.api import PlotCallback


def read_planet_positions(ds):
    data_dir = Path(ds.directory)
    planet_log_files = sorted(data_dir.glob("planet*.dat"))
    nplanets = len(planet_log_files)
    positions = np.empty((nplanets, 4))
    for i, log in enumerate(planet_log_files):
        # load x, y, z, q time columns
        dat = np.loadtxt(log, usecols=(1, 2, 3, 7, 8))
        index = np.argwhere(np.abs(dat[:, -1] - ds.current_time.v) < 0.1).item()
        positions[i] = dat[index, :4].flat
    return positions.T


class Planets(PlotCallback):
    _type_name = "planets"  # -> define a method called `annotate_planets`

    def __init__(self, ds, **kwargs):
        # the __init__ method is responsible for doing expensive tasks
        # but it doesn't actually interact with the plot.
        # The function signature is free.
        self.xp, self.yp, _, qp = read_planet_positions(ds)

        # purely for style, let's scale the marker size with planet mass
        sizes = 100 * np.log10(1 + qp / min(qp))

        self.kwargs = {
            "marker": "x",
            "color": "white",
            "sizes": sizes,
            "linewidths": 3,
            **kwargs,
        }

    def __call__(self, plot) -> None:
        # the __call__ method is responsible for calling matplotlib functions
        # its signature should be exactly this one
        plot._axes.scatter(self.xp, self.yp, **self.kwargs)


p = yt.SlicePlot(ds, "z", "density")
p.annotate_planets(ds)
p.annotate_title("See my planets !")

## 5 Bonus: making 1D profiles

How about 1D reductions ? Let's make an azimuthal average of the density in *one* line of code.

In [ ]:
p = yt.ProfilePlot(ds, "r", "density", x_log=False)
p

If we need to customize the plot, we can do whatever we want by getting back to the fundamental interface (matplotlib). For instance here we'll annotate the radial positions of our planets.

In [ ]:
p = yt.ProfilePlot(ds, "r", "density", x_log=False)

# back to basics: do whatever we want with matplotlib
fig = p.plots["density"].figure
ax = p.plots["density"].axes
xp, yp, _zp, _qp = read_planet_positions(p.ds)
rp = np.sqrt(xp**2 + yp**2)
for x in rp:
    ax.axvline(x, ls="--", color="black", alpha=0.6)
fig